In [4]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import pickle
from keras.utils import np_utils
from tensorflow.keras.layers import InputLayer, Dense, LSTM, Dropout, Activation
from tensorflow.keras.models import Sequential

from music21 import converter, instrument, note, chord, stream
import glob

from keras.utils import np_utils

FONTSIZE=18
plt.rcParams['figure.figsize']=(10,6)
plt.rcParams['font.size']=FONTSIZE

# Load in MIDI files and convert them to notes

In [ ]:
def read_midi(file):
    notes=[]
    notes_to_parse = None

    #parsing a midi file
    midi = converter.parse(file)
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #Looping over all the instruments
    for part in s2.parts:
        #select elements of only piano
        if 'Piano' in str(part): 
            notes_to_parse = part.recurse() 
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
      
    return notes

filenames=glob.glob('../MIDI_files/Classical_Archives_The_Greats/Bach/*.mid')
notes=[]
notes_per_song=[]

for i in range(len(filenames)):
    notes_temp=read_midi(filenames[i])
    notes.append(notes_temp)
    notes_per_song.append(len(notes_temp))

In [5]:
filenames=glob.glob('../MIDI_files/Classical_Archives_The_Greats/Bach/*.mid')

#filename='../MIDI_files/Classical_Archives_The_Greats/Chopin/Prelude n01 op28 \'\'Reunion\'\'.mid'

notes=[]

#this has length num_songs and gives the number of notes in each, this will be useful for partitioning training data
notes_per_song=[]

for i in range(len(filenames)):
    
    file=filenames[i]
    
    print(file)
    
    midi=converter.parse(file)
    
    instr=instrument.Organ #this is the desired individual instrument
    parts=instrument.partitionByInstrument(midi)
    notes_temp=0
    try: #some midis will read in parts as a NoneType object
        
        if len(parts)==1: #song has only one part

            for part in parts:
                if isinstance(part.getInstrument(),instr): #that part is of the desired instrument
                    print("This song has the desired part and only one part")

                    for element in part.flat.notes:

                        if isinstance(element,note.Note):
                            notes.append(str(element.pitch))
                            notes_temp+=1

                        elif isinstance(element,chord.Chord):
                            notes.append('.'.join(str(n) for n in element.normalOrder))
                            notes_temp+=1

                    notes_per_song.append(notes_temp)
        
    except:
        pass #parts is a NoneType object

                    

../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0540 Toccata and Fugue.mid
This song has the desired part and only one part
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0528 Sonate en trio n4.mid
This song has the desired part and only one part
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0997 Partita for Lute 1mov.mid
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0997 Partita for Lute 4mov.mid
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0997 Partita for Lute 3mov.mid
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0806 English Suite n1 02mov .mid
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0535 Prelude and Fugue.mid
This song has the desired part and only one part
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0998 Prelude Fugue Allegro for Lute 3mov.mid
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv1014 Harpsicord and Violin Sonata 3mov.mid
../MIDI_files/Classical_Archives_The_Greats/Bach/Bwv0594 Vivaldi Concerto Arrangement RV20

In [6]:
print(len(notes))
print(sum(notes_per_song))
print(notes_per_song) #what we can actually do here is partition by song
#training data will have roughly 75% of songs and validation data will have remaining songs

num_songs=len(notes_per_song)

78668
78668
[2478, 1624, 886, 5269, 2648, 1209, 2375, 691, 865, 568, 3162, 1958, 2567, 6067, 1012, 2356, 2236, 2834, 4020, 1572, 2373, 1020, 2392, 2142, 2254, 599, 451, 3836, 298, 2574, 540, 1828, 4374, 637, 477, 3096, 2364, 639, 377]


# Remove songs that have less than sequence_length notes

In [7]:
sequence_length=100


#remove songs that have less than sequence_length notes
remove_song_idx=[]
for i in range(num_songs):
    if notes_per_song[i]<sequence_length:
        remove_song_idx.append(i)

print(remove_song_idx)




[]


In [8]:
remove_note_idx=[]
num_note=0

for i in range(num_songs):
    
    num_notes_in_song=notes_per_song[i]

    
    for j in range(num_notes_in_song):
        
        if i in remove_song_idx:
            remove_note_idx.append(num_note)
            num_note+=1
        else:
            num_note+=1
            
print(len(remove_note_idx))
        

0


In [9]:
notes_per_song=np.delete(notes_per_song,remove_song_idx)
notes=np.delete(notes,remove_note_idx)
num_songs=len(notes_per_song)

In [10]:
print(notes_per_song)
print(len(notes))

[2478 1624  886 5269 2648 1209 2375  691  865  568 3162 1958 2567 6067
 1012 2356 2236 2834 4020 1572 2373 1020 2392 2142 2254  599  451 3836
  298 2574  540 1828 4374  637  477 3096 2364  639  377]
78668


# Create ins and outs

In [11]:
# Now what we have to do is create a bunch of sequences of constant length
# we also have to map the notes to integers and then map those integers to categorical variables 
sequence_length=100

#create a list of all pitchnames
pitchnames=set(item for item in notes)

n_notes=len(pitchnames)

#create a dict that maps pitchnames to integers
note_to_int={} #keys are pitchnames, values are integers
count=-1
for i in pitchnames:
    count+=1
    note_to_int[i]=count
    
#calculate total number of sequences
num_sequences=0
for i in range(num_songs):
    num_notes_in_song=notes_per_song[i]
    for j in range(num_notes_in_song-sequence_length):
        num_sequences+=1
    
ins=np.zeros((num_sequences,sequence_length))
outs=[]

#the input is a sequence of sequence_length notes
#the output is the next note after that sequence

note_count=0
sequences_per_song=[]

for i in range(num_songs):
    
    num_notes_in_song=notes_per_song[i]
    sequences_per_song.append(num_notes_in_song-sequence_length)

    
    #here we're grouping it by song, there will not be a single sequence that has part of one song and part of another
    for j in range(num_notes_in_song-sequence_length):
        
        sequence_in=notes[note_count:note_count+sequence_length]
        sequence_out=notes[note_count+sequence_length]
        
        ins_temp=[note_to_int[i] for i in sequence_in]
        
        ins[note_count,:]=ins_temp
        outs.append(note_to_int[sequence_out])
        
        note_count+=1
        
#ins=np.reshape(ins,(num_sequences, sequence_length, 1)) #reshape to keras ready shape
#ins = ins / float(n_notes) #scale to 0-1
#outs=np_utils.to_categorical(outs) #transform outs to a categorical
    

print(sequences_per_song)
print(ins.shape)
print(len(outs))


[2378, 1524, 786, 5169, 2548, 1109, 2275, 591, 765, 468, 3062, 1858, 2467, 5967, 912, 2256, 2136, 2734, 3920, 1472, 2273, 920, 2292, 2042, 2154, 499, 351, 3736, 198, 2474, 440, 1728, 4274, 537, 377, 2996, 2264, 539, 277]
(74768, 100)
74768


# Partition Training and Validation Data

In [12]:
# Ok now we need to define our training and validation sets
# The real challenge for our model is going to be the fact that the validation sequences are from songs that are not included in the training set
# If we can get the model to not overfit we'll have accomplished something pretty impressive

songs_all=np.arange(len(notes_per_song))
idx_train=np.random.choice(songs_all,size=int(len(notes_per_song)*0.75),replace=False) #index of songs used for training
idx_validation=[] #index of songs used for validation
for i in songs_all:
    if i not in idx_train:
        idx_validation.append(i)
        
        
#calculate number of sequences for training and validation
num_sequences_train=0
for i in idx_train:
    num_sequences_train+=sequences_per_song[i]
num_sequences_validation=sum(sequences_per_song)-num_sequences_train

print("Number of Training Sequences is "+str(num_sequences_train))
print("Number of Validation Sequences is "+str(num_sequences_validation))
        
ins_train=np.zeros((num_sequences_train,sequence_length))
outs_train=[]
ins_validation=np.zeros((num_sequences_validation,sequence_length))
outs_validation=[]
        
idx_sequence=-1
idx_train_sequence=-1
idx_validation_sequence=-1
for i in range(num_songs):
    
    sequences_in_song=sequences_per_song[i]
    
    for j in range(sequences_in_song):
        idx_sequence+=1
        
        if i in idx_train:
            idx_train_sequence+=1
            ins_train[idx_train_sequence,:]=ins[idx_sequence,:]
            outs_train.append(outs[idx_sequence])
            
            
        elif i in idx_validation:
            idx_validation_sequence+=1
            ins_validation[idx_validation_sequence,:]=ins[idx_sequence,:]
            outs_validation.append(outs[idx_sequence])
    

ins_train=np.reshape(ins_train,(num_sequences_train,sequence_length,1)) #reshape to keras ready shape
ins_validation=np.reshape(ins_validation,(num_sequences_validation,sequence_length,1))

ins_train = ins_train / float(n_notes) #scale to 0-1
ins_validation=ins_validation/float(n_notes)

outs_train=np_utils.to_categorical(outs_train) #transform outs to a categorical
outs_validation=np_utils.to_categorical(outs_validation)

print("ins_train shape: "+str(ins_train.shape))
print("outs_train shape: "+str(outs_train.shape))
print("ins_validation shape: "+str(ins_validation.shape))
print("outs_validation shape: "+str(outs_validation.shape))

Number of Training Sequences is 48284
Number of Validation Sequences is 26484
ins_train shape: (48284, 100, 1)
outs_train shape: (48284, 223)
ins_validation shape: (26484, 100, 1)
outs_validation shape: (26484, 223)


In [13]:
training_data={'ins_train':ins_train,'outs_train':outs_train,'ins_validation':ins_validation,'outs_validation':outs_validation}

#fp=open("training_data_bach.pkl",'wb')
#pickle.dump(training_data,fp)
#fp.close()

In [14]:
print(ins_train[0,:,0])

[0.37219731 0.53363229 0.37219731 0.15246637 0.39910314 0.37219731
 0.77130045 0.37219731 0.77130045 0.15246637 0.0941704  0.77130045
 0.39910314 0.37219731 0.03587444 0.79820628 0.93721973 0.74887892
 0.93721973 0.03587444 0.93721973 0.03587444 0.95515695 0.74887892
 0.39910314 0.0941704  0.39910314 0.0941704  0.77130045 0.95515695
 0.77130045 0.39910314 0.0941704  0.39910314 0.77130045 0.37219731
 0.39910314 0.77130045 0.39910314 0.77130045 0.37219731 0.53363229
 0.77130045 0.37219731 0.53363229 0.37219731 0.77130045 0.39910314
 0.37219731 0.77130045 0.37219731 0.77130045 0.39910314 0.0941704
 0.77130045 0.39910314 0.77130045 0.39910314 0.0941704  0.95515695
 0.39910314 0.0941704  0.39910314 0.0941704  0.95515695 0.74887892
 0.0941704  0.95515695 0.0941704  0.95515695 0.37219731 0.04932735
 0.95515695 0.74887892 0.95515695 0.74887892 0.37219731 0.03587444
 0.74887892 0.04932735 0.74887892 0.04932735 0.03587444 0.51569507
 0.95067265 0.37668161 0.95067265 0.37668161 0.80269058 0.95067

In [ ]:
index=np.arange(sequence_length)
plt.plot(index,ins_train[0,:,0])

In [15]:
model = Sequential()
model.add(LSTM(256,
               input_shape=(sequence_length, 1),
               return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(outs_train.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [16]:
# Callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                  min_delta=0.01)

In [ ]:
history=model.fit(ins_train, 
                  outs_train,
                  validation_data=(ins_validation, outs_validation),
                  epochs=10, 
                  batch_size=64,
                  callbacks=[early_stopping_cb])

Train on 48284 samples, validate on 26484 samples
Epoch 1/10
48284/48284 [==============================] - 1135s 24ms/sample - loss: 3.9176 - categorical_accuracy: 0.0618 - val_loss: 3.8301 - val_categorical_accuracy: 0.0700
Epoch 2/10
48284/48284 [==============================] - 1176s 24ms/sample - loss: 3.8734 - categorical_accuracy: 0.0681 - val_loss: 3.8290 - val_categorical_accuracy: 0.0700
Epoch 3/10
48284/48284 [==============================] - 1769s 37ms/sample - loss: 3.8725 - categorical_accuracy: 0.0655 - val_loss: 3.8233 - val_categorical_accuracy: 0.0700
Epoch 4/10
48284/48284 [==============================] - 1198s 25ms/sample - loss: 3.8718 - categorical_accuracy: 0.0652 - val_loss: 3.8215 - val_categorical_accuracy: 0.0700
Epoch 5/10
21056/48284 [============>.................] - ETA: 9:06 - loss: 3.8743 - categorical_accuracy: 0.0655

In [ ]:
epochs=np.arange(len(history.history['loss']))

plt.plot(epochs,history.history['categorical_accuracy'],label='Train')
plt.plot(epochs,history.history['val_categorical_accuracy'],label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Categorical Accuracy')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(epochs,history.history['loss'],label='Train')
plt.plot(epochs,history.history['val_loss'],label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Categorical Crossentropy')
plt.legend(loc='best')
plt.show()